题目：编写一个 Python 脚本，让一个球在某个形状内弹跳。让该形状缓慢旋转，并确保球停留在形状内。

In [ ]:
# Gemini Pro 2.0
import pygame
import math

# 初始化 Pygame
pygame.init()

# 设置屏幕尺寸
screen_width = 800
screen_height = 600
screen = pygame.display.set_mode((screen_width, screen_height))

# 定义颜色
white = (255, 255, 255)
black = (0, 0, 0)
red = (255, 0, 0)

# 球体参数
ball_x = screen_width // 2
ball_y = screen_height // 2
ball_radius = 15
ball_speed_x = 5
ball_speed_y = 5

# 形状参数 (这里使用一个矩形,您可以轻松修改为其他形状)
shape_x = screen_width // 4
shape_y = screen_height // 4
shape_width = screen_width // 2
shape_height = screen_height // 2
shape_rotation = 0  # 初始旋转角度
rotation_speed = 0.1  # 旋转速度


# 游戏循环
running = True
clock = pygame.time.Clock()

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # 更新球体位置
    ball_x += ball_speed_x
    ball_y += ball_speed_y

    # 旋转形状 (围绕中心)
    shape_rotation = (shape_rotation + rotation_speed) % 360
    shape_center_x = shape_x + shape_width / 2
    shape_center_y = shape_y + shape_height / 2

    # 创建旋转后的矩形
    rotated_rect = pygame.Rect(0, 0, shape_width, shape_height)
    rotated_rect.center = (shape_center_x, shape_center_y)

    # 计算四个角的坐标(旋转后)
    angle_rad = math.radians(shape_rotation)
    corners = []
    for x_off, y_off in [
        (rotated_rect.left - shape_center_x, rotated_rect.top - shape_center_y),
        (rotated_rect.right - shape_center_x, rotated_rect.top - shape_center_y),
        (rotated_rect.right - shape_center_x, rotated_rect.bottom - shape_center_y),
        (rotated_rect.left - shape_center_x, rotated_rect.bottom - shape_center_y),
    ]:
        x = shape_center_x + x_off * math.cos(angle_rad) - y_off * math.sin(angle_rad)
        y = shape_center_y + x_off * math.sin(angle_rad) + y_off * math.cos(angle_rad)
        corners.append((x, y))

    # 碰撞检测与响应 (更精确的方法,适用于旋转矩形)
    def point_in_rotated_rect(point, rect_center, width, height, angle):
        # 将点坐标转换到矩形局部坐标系
        x, y = point
        cx, cy = rect_center
        angle_rad = -math.radians(angle)  # 旋转角度取负,因为我们希望点相对矩形旋转

        rotated_x = (x - cx) * math.cos(angle_rad) - (y - cy) * math.sin(angle_rad)
        rotated_y = (x - cx) * math.sin(angle_rad) + (y - cy) * math.cos(angle_rad)

        # 在局部坐标系中进行 AABB 碰撞检测
        return (
            -width / 2 <= rotated_x <= width / 2
            and -height / 2 <= rotated_y <= height / 2
        )

    if not point_in_rotated_rect(
        (ball_x, ball_y),
        (shape_center_x, shape_center_y),
        shape_width,
        shape_height,
        shape_rotation,
    ):
        # 找出球心到矩形四个边距离最近的边
        distances = []
        # 遍历矩形的四个边
        for i in range(4):
            x1, y1 = corners[i]
            x2, y2 = corners[(i + 1) % 4]  # 循环访问四个点

            # 计算点到直线的距离,使用向量方法
            dx = x2 - x1
            dy = y2 - y1

            if dx == 0 and dy == 0:  # 处理两个点重合的情况(理论上不会发生,但以防万一)
                dist = math.sqrt((ball_x - x1) ** 2 + (ball_y - y1) ** 2)
            else:
                t = ((ball_x - x1) * dx + (ball_y - y1) * dy) / (
                    dx**2 + dy**2
                )  # 投影长度
                t = max(0, min(1, t))  # 钳制 t 到 [0, 1],确保投影点在线段上

                closest_x = x1 + t * dx
                closest_y = y1 + t * dy
                dist = math.sqrt((ball_x - closest_x) ** 2 + (ball_y - closest_y) ** 2)

            distances.append((dist, (x1, y1), (x2, y2)))  # 存储距离和对应的边

        distances.sort()  # 按距离排序
        min_dist, (x1, y1), (x2, y2) = distances[0]  # 最短的距离

        # 使用法线向量进行反弹
        dx = x2 - x1
        dy = y2 - y1
        normal_x = -dy
        normal_y = dx

        # 归一化法向量
        length = math.sqrt(normal_x**2 + normal_y**2)
        if length != 0:
            normal_x /= length
            normal_y /= length

        # 计算点积
        dot_product = ball_speed_x * normal_x + ball_speed_y * normal_y

        # 根据法线向量反转速度
        ball_speed_x -= 2 * dot_product * normal_x
        ball_speed_y -= 2 * dot_product * normal_y

        # 将小球稍微推离边界,防止粘滞现象
        epsilon = 2
        ball_x += normal_x * epsilon
        ball_y += normal_y * epsilon

    # 清除屏幕
    screen.fill(black)

    # 绘制旋转后的矩形
    pygame.draw.polygon(screen, white, corners)

    # 绘制球体
    pygame.draw.circle(screen, red, (int(ball_x), int(ball_y)), ball_radius)

    # 更新显示
    pygame.display.flip()

    # 控制帧率
    clock.tick(60)

pygame.quit()

In [ ]:
# DeepSeek R1
import pygame
import math
import sys

# 初始化 Pygame
pygame.init()
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()

# 颜色定义
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)

# 多边形参数
NUM_SIDES = 5  # 五边形
RADIUS = 250  # 多边形半径
ROTATION_SPEED = 0.5  # 旋转速度(度/帧)
rotation_angle = 0  # 初始旋转角度

# 小球参数
BALL_RADIUS = 10
ball_pos = [WIDTH // 2, HEIGHT // 2]
ball_velocity = [3, -5]  # 小球初始速度


def generate_polygon_points(rotation):
    """生成旋转后的多边形顶点坐标"""
    points = []
    angle_step = 360 / NUM_SIDES
    for i in range(NUM_SIDES):
        angle_rad = math.radians(i * angle_step + rotation)
        x = WIDTH // 2 + RADIUS * math.cos(angle_rad)
        y = HEIGHT // 2 + RADIUS * math.sin(angle_rad)
        points.append((x, y))
    return points


def is_point_inside_polygon(point, polygon):
    """射线法判断点是否在多边形内部"""
    x, y = point
    inside = False
    n = len(polygon)
    for i in range(n):
        x1, y1 = polygon[i]
        x2, y2 = polygon[(i + 1) % n]
        if ((y1 > y) != (y2 > y)) and (x < (x2 - x1) * (y - y1) / (y2 - y1) + x1):
            inside = not inside
    return inside


def reflect_vector(vec, edge):
    """根据边的法线反射速度向量"""
    a, b = edge
    edge_vec = (b[0] - a[0], b[1] - a[1])
    normal = (-edge_vec[1], edge_vec[0])  # 边的外法线方向

    # 归一化法线
    length = math.hypot(normal[0], normal[1])
    if length == 0:
        return vec
    normal = (normal[0] / length, normal[1] / length)

    dot = vec[0] * normal[0] + vec[1] * normal[1]
    return [vec[0] - 2 * dot * normal[0], vec[1] - 2 * dot * normal[1]]


# 主循环
running = True
polygon = []  # 保存多边形顶点

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    screen.fill(WHITE)

    # 更新多边形旋转
    rotation_angle = (rotation_angle + ROTATION_SPEED) % 360
    polygon = generate_polygon_points(rotation_angle)
    pygame.draw.polygon(screen, BLACK, polygon, 2)

    # 更新小球位置
    proposed_pos = [ball_pos[0] + ball_velocity[0], ball_pos[1] + ball_velocity[1]]

    # 碰撞检测
    if is_point_inside_polygon(proposed_pos, polygon):
        ball_pos = proposed_pos
    else:
        # 找到反射边并调整速度
        closest_edge = None
        min_distance = float("inf")

        for i in range(len(polygon)):
            a = polygon[i]
            b = polygon[(i + 1) % len(polygon)]
            # 计算检测点到此边的最短距离
            edge_vec = (b[0] - a[0], b[1] - a[1])
            point_vec = (proposed_pos[0] - a[0], proposed_pos[1] - a[1])
            t = max(
                0,
                min(
                    1,
                    (point_vec[0] * edge_vec[0] + point_vec[1] * edge_vec[1])
                    / (edge_vec[0] ** 2 + edge_vec[1] ** 2 + 1e-6),
                ),
            )
            projection = (a[0] + t * edge_vec[0], a[1] + t * edge_vec[1])
            distance = math.hypot(
                proposed_pos[0] - projection[0], proposed_pos[1] - projection[1]
            )

            if distance < min_distance:
                min_distance = distance
                closest_edge = (a, b)

        # 反射速度向量
        if closest_edge:
            ball_velocity = reflect_vector(ball_velocity, closest_edge)

    # 更新小球位置
    ball_pos[0] += ball_velocity[0]
    ball_pos[1] += ball_velocity[1]

    # 绘制小球
    pygame.draw.circle(screen, RED, (int(ball_pos[0]), int(ball_pos[1])), BALL_RADIUS)

    pygame.display.flip()
    clock.tick(60)

pygame.quit()
sys.exit()


题目：write a Python program that shows a ball bouncing inside a spinning hexagon. The ball should be affected by gravity and friction, and it must bounce off the rotating walls realistically

In [ ]:
# Gemini Pro 2.0
import pygame
import math

# --- Constants ---
WIDTH, HEIGHT = 800, 600
BALL_RADIUS = 15
HEXAGON_RADIUS = 200
GRAVITY = 0.2  # Adjust for stronger/weaker gravity
FRICTION = 0.98  # Energy loss on each bounce (0.99 = less loss)
ROTATION_SPEED = 0.01  # radians per frame

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)

# --- Functions ---


def rotate_point(point, center, angle):
    """Rotates a point around a center by a given angle."""
    x, y = point
    cx, cy = center
    rotated_x = (x - cx) * math.cos(angle) - (y - cy) * math.sin(angle) + cx
    rotated_y = (x - cx) * math.sin(angle) + (y - cy) * math.cos(angle) + cy
    return (rotated_x, rotated_y)


def point_to_line_distance(point, line_start, line_end):
    """Calculates the distance from a point to a line segment."""
    x0, y0 = point
    x1, y1 = line_start
    x2, y2 = line_end

    # Calculate length of the line segment
    line_length = math.dist((x1, y1), (x2, y2))

    # If line segment is a point
    if line_length < 0.0001:  # Use a small tolerance to avoid division by zero
        return math.dist(point, line_start)

    # Project the vector (p-v) onto w.
    dx, dy = x2 - x1, y2 - y1
    t = ((x0 - x1) * dx + (y0 - y1) * dy) / (dx**2 + dy**2)

    # Clamp t so that projected point lands on the line segment.
    t = max(0, min(1, t))

    # Calculate the closest point
    closest_x = x1 + t * dx
    closest_y = y1 + t * dy
    distance = math.dist((x0, y0), (closest_x, closest_y))

    return distance


def get_normal_vector(line_start, line_end):
    """Calculates the unit normal vector to a line segment."""
    x1, y1 = line_start
    x2, y2 = line_end
    dx = x2 - x1
    dy = y2 - y1
    # Normal vector is perpendicular to the line, so we swap dx and dy, and negate one
    normal_x, normal_y = -dy, dx
    # Normalize the vector
    length = math.sqrt(normal_x**2 + normal_y**2)
    if length != 0:
        normal_x /= length
        normal_y /= length
    return normal_x, normal_y


# --- Initialize Pygame ---
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Bouncing Ball in Rotating Hexagon")
clock = pygame.time.Clock()


# --- Game Variables ---
ball_x = WIDTH // 2
ball_y = HEIGHT // 2 - 50
ball_vx = 0
ball_vy = 0
hexagon_angle = 0

# --- Main Game Loop ---
running = True
while running:
    # Event Handling
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # --- Update Game State ---

    # * Gravity
    ball_vy += GRAVITY

    # * Update Ball Position
    ball_x += ball_vx
    ball_y += ball_vy

    # * Rotate Hexagon
    hexagon_angle += ROTATION_SPEED  # Keep angle within 0-2pi for efficiency
    hexagon_angle %= 2 * math.pi

    # * Calculate Hexagon Vertices (rotated)
    hexagon_center = (WIDTH // 2, HEIGHT // 2)
    hexagon_vertices = []
    for i in range(6):
        angle = (
            i * (math.pi / 3) + hexagon_angle
        )  # 60 degrees (pi/3) increments, plus rotation
        x = hexagon_center[0] + HEXAGON_RADIUS * math.cos(angle)
        y = hexagon_center[1] + HEXAGON_RADIUS * math.sin(angle)
        hexagon_vertices.append((x, y))

    # * Collision Detection and Response
    for i in range(6):
        start_vertex = hexagon_vertices[i]
        end_vertex = hexagon_vertices[(i + 1) % 6]
        distance = point_to_line_distance((ball_x, ball_y), start_vertex, end_vertex)

        if distance < BALL_RADIUS:
            # Collision!
            normal_x, normal_y = get_normal_vector(start_vertex, end_vertex)

            # Reflect velocity across the normal
            dot_product = ball_vx * normal_x + ball_vy * normal_y
            ball_vx -= 2 * dot_product * normal_x
            ball_vy -= 2 * dot_product * normal_y

            # * Friction (apply after reflection)
            ball_vx *= FRICTION
            ball_vy *= FRICTION

            # * Push the ball out of the wall (avoid sticking)
            overlap = BALL_RADIUS - distance
            ball_x += normal_x * overlap
            ball_y += normal_y * overlap

    # --- Drawing ---
    screen.fill(BLACK)

    # Draw the hexagon
    pygame.draw.polygon(screen, WHITE, hexagon_vertices, 2)

    # Draw the ball
    pygame.draw.circle(screen, RED, (int(ball_x), int(ball_y)), BALL_RADIUS)

    pygame.display.flip()
    clock.tick(60)  # Limit to 60 frames per second

pygame.quit()

In [ ]:
# DeepSeek R1
import pygame
import math
import sys

# 初始化 Pygame
pygame.init()

# 窗口尺寸
WIDTH = 800
HEIGHT = 600
CENTER = (WIDTH // 2, HEIGHT // 2)

# 颜色定义
WHITE = (255, 255, 255)
RED = (255, 0, 0)
BLUE = (0, 0, 255)

# 物理参数
GRAVITY = 0.2
FRICTION = 0.999
BOUNCE = 0.8
ROTATION_SPEED = 1  # 六边形旋转速度 (度/帧)
WALL_ROTATION_INFLUENCE = 0.1  # 墙面旋转对碰撞的影响系数


class Hexagon:
    def __init__(self, radius=200):
        self.radius = radius
        self.angle = 0
        self.edges = []
        self.update_edges()

    def update_edges(self):
        angle_step = 60
        points = []
        for i in range(6):
            theta = math.radians(self.angle + angle_step * i)
            x = CENTER[0] + self.radius * math.cos(theta)
            y = CENTER[1] + self.radius * math.sin(theta)
            points.append((x, y))

        self.edges = []
        for i in range(6):
            p1 = points[i]
            p2 = points[(i + 1) % 6]

            # 计算边法线方向
            dx = p2[0] - p1[0]
            dy = p2[1] - p1[1]
            normal = (-dy, dx)
            length = math.hypot(*normal)
            normal = (normal[0] / length, normal[1] / length)

            self.edges.append((p1, p2, normal))


class Ball:
    def __init__(self):
        self.radius = 15
        self.x = CENTER[0]
        self.y = CENTER[1]
        self.vx = 5
        self.vy = -4

    def update(self):
        # 应用重力
        self.vy += GRAVITY

        # 应用空气摩擦
        self.vx *= FRICTION
        self.vy *= FRICTION

        # 更新位置
        self.x += self.vx
        self.y += self.vy

    def draw(self, surface):
        pygame.draw.circle(surface, RED, (int(self.x), int(self.y)), self.radius)


def closest_point_on_segment(p, a, b):
    ap = (p[0] - a[0], p[1] - a[1])
    ab = (b[0] - a[0], b[1] - a[1])
    t = (ap[0] * ab[0] + ap[1] * ab[1]) / (ab[0] ** 2 + ab[1] ** 2 + 1e-8)  # 防止除以零
    t = max(0, min(1, t))
    return (a[0] + ab[0] * t, a[1] + ab[1] * t)


def check_collision(ball, hexagon):
    collision_occurred = False
    for edge in hexagon.edges:
        p1, p2, normal = edge

        # 找到离球最近的边上点
        closest = closest_point_on_segment((ball.x, ball.y), p1, p2)

        # 计算距离向量kll
        dx = ball.x - closest[0]
        dy = ball.y - closest[1]
        distance = math.hypot(dx, dy)

        if distance < ball.radius:
            # 计算墙面旋转带来的速度影响
            center_to_closest = (closest[0] - CENTER[0], closest[1] - CENTER[1])
            wall_velocity = (
                -center_to_closest[1] * math.radians(ROTATION_SPEED),
                center_to_closest[0] * math.radians(ROTATION_SPEED),
            )

            # 计算相对于墙面的速度
            relative_vx = ball.vx - wall_velocity[0] * WALL_ROTATION_INFLUENCE
            relative_vy = ball.vy - wall_velocity[1] * WALL_ROTATION_INFLUENCE

            # 计算法向速度
            dot_product = relative_vx * normal[0] + relative_vy * normal[1]

            if dot_product < 0:
                # 反射速度矢量
                ball.vx -= 2 * dot_product * normal[0]
                ball.vy -= 2 * dot_product * normal[1]

                # 应用能量损失
                ball.vx *= BOUNCE
                ball.vy *= BOUNCE

                # 防止球嵌入墙面
                overlap = ball.radius - distance
                ball.x += normal[0] * overlap
                ball.y += normal[1] * overlap

                collision_occurred = True

    return collision_occurred


# 初始化
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("旋转六边形弹球")
clock = pygame.time.Clock()

hexagon = Hexagon()
ball = Ball()

# 主循环
running = True
while running:
    # 处理事件
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # 更新状态
    hexagon.angle = (hexagon.angle + ROTATION_SPEED) % 360
    hexagon.update_edges()

    # 更新球位置
    ball.update()

    # 碰撞检测
    check_collision(ball, hexagon)

    # 绘制画面
    screen.fill(WHITE)

    # 绘制六边形
    for edge in hexagon.edges:
        pygame.draw.line(screen, BLUE, edge[0], edge[1], 3)

    ball.draw(screen)

    # 刷新显示
    pygame.display.flip()
    clock.tick(60)

pygame.quit()
sys.exit()
